In [35]:
import re
import random
import pandas as pd
import numpy as np

In [2]:
# Removes everything except letters, apostrophes, whitespaces
def clean_articles(text):
    return ' '.join(re.sub('[^a-zA-Z\' ]', ' ', text).split()).lower()

# Tokenizes the text into trigrams
def tokenize(text):
    tokens = text.split()
    num_words = len(tokens)
    tokens_set = []
    for num in range(0, num_words - 2):
        word = tokens[num] + " " + tokens[num + 1] + " " + tokens[num + 2]
        tokens_set.append(word)
    return tokens_set

In [4]:
# Read the articles, cleans and tokenize the text
train_set = 'data/articles_100.train'
with open(train_set, 'r') as f:
    data = f.read().split('\n')

data_articles = {}
for line in data:
    if line:
        newline = line.split(' ', 1)
        data_articles[newline[0]] = tokenize(clean_articles(newline[1]))
print 'Number of articles in training set:', len(data_articles)
del line
del newline
del data

Number of articles in training set: 100


In [5]:
# Indexes all the trigrams so that each trigram is represented by a unique integer.
#word_set is {word : index}
word_set = {}
count = 1
for key, value in data_articles.iteritems():
    for word in value:
        if word not in word_set:
            word_set[word] = count
            count = count + 1
del count
del word
del key
del value

In [6]:
# Returns a list of size num_features, where the elements are random integers between 0 and max_shingle_size
def pick_rand_coeff(max_shingle_size, num_features):
    randlist = []
    assert num_features < max_shingle_size, 'Feature size must be less than max shingle size'
    for i in range(0, num_features):
        rand_gen_int = random.randint(0, max_shingle_size)
        while rand_gen_int in randlist:
            rand_gen_int = random.randint(0, max_shingle_size)
        randlist.append(rand_gen_int)
    return randlist

# Computes the hash function using the formula (ax + b) % c, where a and b are randomly generated integers and c is a 
# prime number slightly larger than max_shingle_size
def hash_function(x, coef_a, coef_b, prime):
    return (x * coef_a + coef_b) % prime

In [7]:
# Creates the arrays for a and b
arr_coeff_a = pick_rand_coeff(len(word_set), 10)
arr_coeff_b = pick_rand_coeff(len(word_set), 10)

In [8]:
# Calculates the minhashes for each document
prime = 22003
data_hashes = []
for key, value in data_articles.iteritems():
    signature = [key]
    for hash_index in range(0, 10):
        minimum = prime
        for word in value:
            hash_value = hash_function(word_set[word], arr_coeff_a[hash_index], arr_coeff_b[hash_index], prime)
            if hash_value < minimum:
                minimum = hash_value
        signature.append(minimum)
    data_hashes.append(signature)
del key
del value
del signature
del hash_index
del minimum
del word
del hash_value

In [9]:
# Compares the minhashes between the documents and flags out similar documents based on the minhash
threshold = 5
results = []
for i in range(0, len(data_hashes)):
    key1 = data_hashes[i][0]
    signature1 = data_hashes[i][1:]
    
    for j in range(i + 1, len(data_hashes)):
        key2 = data_hashes[j][0]
        signature2 = data_hashes[j][1:]
    
        count = 0
        for signature in signature1:
            if signature in signature2:
                count = count + 1
        if count > threshold:
            if int(key1[1:]) > int(key2[1:]):
                results.append(key2 + ' ' + key1)
                print 'Document 1:', key2, '\nDocument 2:', key1, '\nCount of similar items:', count, '\n'
            else:
                results.append(key1 + ' ' + key2)
                print 'Document 1:', key1, '\nDocument 2:', key2, '\nCount of similar items:', count, '\n'

Document 1: t1952 
Document 2: t3495 
Count of similar items: 10 

Document 1: t1768 
Document 2: t5248 
Count of similar items: 9 

Document 1: t1088 
Document 2: t5015 
Count of similar items: 10 

Document 1: t980 
Document 2: t2023 
Count of similar items: 10 

Document 1: t1297 
Document 2: t4638 
Count of similar items: 10 



In [10]:
train_set = 'data/articles_100.truth'
with open(train_set, 'r') as f:
    data = f.read().split('\n')

data_truth = []
for pair in data:
    if pair:
        if int(pair.split()[0][1:]) > int(pair.split()[1][1:]):
            data_truth.append(pair.split()[1] + ' ' + pair.split()[0])
            print 'Document 1:', pair.split()[1], '\nDocument 2:', pair.split()[0], '\n'
        else:
            data_truth.append(pair.split()[0] + ' ' + pair.split()[1])
            print 'Document 1:', pair.split()[0], '\nDocument 2:', pair.split()[1], '\n'

Document 1: t1088 
Document 2: t5015 

Document 1: t1297 
Document 2: t4638 

Document 1: t1768 
Document 2: t5248 

Document 1: t1952 
Document 2: t3495 

Document 1: t980 
Document 2: t2023 



In [11]:
count = 0
for result in results:
    if result in data_truth:
        count = count + 1
print 'Accuracy:', count / float(len(data_truth)) * 100, '%'

Accuracy: 100.0 %


Permutation

In [67]:
matrix_article = pd.DataFrame([])
for key, article in data_articles.iteritems():
    row_article = pd.DataFrame(np.zeros((1, len(matrix_article.columns))), columns = matrix_article.columns)
    for word in article:
        if word not in matrix_article.columns:
            matrix_article[word] = 0
        row_article[word] = 1
    matrix_article = pd.concat([matrix_article, row_article])